In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 88 kB/s 
     |████████████████████████████████| 198 kB 58.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=0e928c8f788e469b360ecaa4eac3ecc68f5e5d9a3037e3ae3818171922736070
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

21/08/17 06:24:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стоимость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стоимость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [28]:
schema_df = StructType(
    [
        StructField('datesold', DateType(), True),
        StructField('postcode', IntegerType(), True),
        StructField('price', IntegerType(), True),
        StructField('propertyType', StringType(), True),
        StructField('bedrooms', IntegerType(), True),
    ]
)

df = spark.read.format('csv').schema(schema_df).option('header', 'True').load('raw_sales.csv')

In [29]:
#df.printSchema()
df.registerTempTable('df')
df.show()

+----------+--------+-------+------------+--------+
|  datesold|postcode|  price|propertyType|bedrooms|
+----------+--------+-------+------------+--------+
|2007-02-07|    2607| 525000|       house|       4|
|2007-02-27|    2906| 290000|       house|       3|
|2007-03-07|    2905| 328000|       house|       3|
|2007-03-09|    2905| 380000|       house|       4|
|2007-03-21|    2906| 310000|       house|       3|
|2007-04-04|    2905| 465000|       house|       4|
|2007-04-24|    2607| 399000|       house|       3|
|2007-04-30|    2606|1530000|       house|       4|
|2007-05-24|    2902| 359000|       house|       3|
|2007-05-25|    2906| 320000|       house|       3|
|2007-06-26|    2902| 385000|       house|       3|
|2007-06-27|    2906| 305000|       house|       3|
|2007-06-27|    2612| 850000|       house|       4|
|2007-06-28|    2904| 765000|       house|       4|
|2007-06-30|    2615| 517000|       house|       4|
|2007-07-02|    2914| 800000|       house|       5|
|2007-07-03|

In [33]:
from pyspark.sql.window import Window

windSpec = Window.partitionBy('postcode').orderBy('datesold').rowsBetween(-10, Window.currentRow)
df.withColumn('avg_pr_before', F.round(F.avg('price').over(windSpec), 2)).show()

+----------+--------+------+------------+--------+-------------+
|  datesold|postcode| price|propertyType|bedrooms|avg_pr_before|
+----------+--------+------+------------+--------+-------------+
|2007-07-02|    2914|800000|       house|       5|     800000.0|
|2008-06-17|    2914|600000|       house|       4|     700000.0|
|2008-08-29|    2914|465000|       house|       4|    621666.67|
|2008-09-02|    2914|541000|       house|       4|     601500.0|
|2008-09-05|    2914|395000|       house|       3|     560200.0|
|2008-09-05|    2914|552000|       house|       4|    558833.33|
|2008-09-17|    2914|410000|       house|       3|    537571.43|
|2008-09-26|    2914|755000|       house|       4|     564750.0|
|2008-10-14|    2914|420000|       house|       4|    548666.67|
|2008-10-16|    2914|375000|       house|       3|     531300.0|
|2008-10-21|    2914|515000|       house|       4|    529818.18|
|2008-10-27|    2914|440000|       house|       3|    497090.91|
|2008-10-27|    2914|4750

In [34]:
windSpec = Window.partitionBy('postcode').orderBy('datesold').rowsBetween(Window.currentRow, 10)
df.withColumn('avg_pr_after', F.round(F.avg('price').over(windSpec), 2)).show()

+----------+--------+------+------------+--------+------------+
|  datesold|postcode| price|propertyType|bedrooms|avg_pr_after|
+----------+--------+------+------------+--------+------------+
|2007-07-02|    2914|800000|       house|       5|   529818.18|
|2008-06-17|    2914|600000|       house|       4|   497090.91|
|2008-08-29|    2914|465000|       house|       4|   485727.27|
|2008-09-02|    2914|541000|       house|       4|   486863.64|
|2008-09-05|    2914|395000|       house|       3|   486772.73|
|2008-09-05|    2914|552000|       house|       4|   505409.09|
|2008-09-17|    2914|410000|       house|       3|   496681.82|
|2008-09-26|    2914|755000|       house|       4|   499772.73|
|2008-10-14|    2914|420000|       house|       4|    467500.0|
|2008-10-16|    2914|375000|       house|       3|   466590.91|
|2008-10-21|    2914|515000|       house|       4|   483409.09|
|2008-10-27|    2914|440000|       house|       3|   489772.73|
|2008-10-27|    2914|475000|       house

In [35]:
windSpec = Window.partitionBy('postcode').orderBy('datesold').rowsBetween(Window.currentRow, Window.unboundedFollowing)


## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [19]:
df_year_price = df.groupBy(F.year('datesold')).agg(F.round(F.avg(F.col('price')), 2).alias('avg_price'))
df_year_price.show()

+--------------+---------+
|year(datesold)|avg_price|
+--------------+---------+
|          2007|522377.21|
|          2018|660701.04|
|          2015|626101.34|
|          2013| 553416.3|
|          2014|592653.76|
|          2019|634184.22|
|          2012|552501.37|
|          2009|496092.03|
|          2016|635185.31|
|          2010|559564.81|
|          2011|566715.11|
|          2008|493814.16|
|          2017|671880.55|
+--------------+---------+



## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)